# Extract Plaid Asset Report based on report token
## Last Updated: 2022-05-31

### Reference
1. Using the customer_plaid_asset_report_ids.csv file to extract asset report token
2. This script use environment variables to hide client ID and secret, referenced from https://dev.to/jakewitcher/using-env-files-for-environment-variables-in-python-applications-55a1

---

### Code

In [1]:
# basic libraries
import requests
import json
from datetime import date
from datetime import datetime
import pandas as pd
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path('myplaid.env')
load_dotenv(dotenv_path=dotenv_path)

client_id = os.getenv('PLAID_CLIENT_ID')
secret = os.getenv('PLAID_SECRET')

In [2]:
# Extracting Asset report ID and token

## read file
customer_assetreport = pd.read_csv("customer_plaid_asset_report_ids.csv", encoding = 'utf-8')

## filter dataframe with valid asset report token 
customer_assetreport = customer_assetreport[customer_assetreport['Asset report token'].str.contains("production")] 

## make local directory and path variable to store files
isExist = os.path.exists("AssetReports")
if not isExist:
    os.mkdir("AssetReports")
path = "AssetReports/"

In [3]:
# Set request URL and variable
url = "https://production.plaid.com/asset_report/get"
asset_report_token = ''

In [6]:
# Set functions to send request and parse response
def assetreport_request(url,client_id,secret,asset_report_token):
    payload = json.dumps({
      "client_id": client_id,
      "secret": secret,
      "asset_report_token": asset_report_token
    })
    headers = {
      'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    return response
def writereport_json(path,response,asset_report_token):
    report_data = response.json()['report']
    json_string = json.dumps(report_data,default=str)
    with open(path+asset_report_token+'.json', 'w') as outfile:
        outfile.write(json_string)

In [7]:
# iterate customer_assetreport to get reports into JSON files
## now only set to take the top three rows of the client data
for index, row in customer_assetreport[:3].iterrows():
    asset_report_token = row['Asset report token']
    response = assetreport_request(url,client_id,secret,asset_report_token)
    writereport_json(path,response,asset_report_token)